## DSI-06 Homework 4: ANSWERS
From Chapter 5, found on pages 225-226 of ISLP

*In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the `Default` data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.*

In [ ]:
# Import standard libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import statsmodels.api as sm
import random

# Import specific objects
from ISLP import load_data
from ISLP import confusion_table
from ISLP.models import contrast
from ISLP.models import (ModelSpec as MS,
                         summarize)
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Load Default data
Default = load_data('Default')
Default

(a) Fit a logistic regression model that uses income and balance to predict default.

In [ ]:
# Convert 'default' to numeric (1 or 0)
Default['default_numeric'] = Default['default'].map({'Yes': 1, 'No': 0})

# Set random seed (use any number you wish)
random_seed = 16 
np.random.seed(random_seed)
random.seed(random_seed)

# Fit logistic regression model
formula = 'default_numeric ~ income + balance'
glm_model = sm.Logit.from_formula(formula, data=Default)
glm_fit = glm_model.fit()

# Print summary
print(glm_fit.summary())

The standard errors of the coefficients are listed in the table above as std err (Income -> 4.99e-06, Balance
-> 0.000)

Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

(i) Split the sample set into a training set and a validation set.

(ii)  Fit a multiple logistic regression model using only the training observations.

(iii)  Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

(vi)  Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [ ]:
# Step (i): Split the dataset into training and validation sets
train_df, valid_df = train_test_split(Default, test_size=0.2, random_state=16)

# Step (ii): Fit a logistic regression model using only the training observations
X_train = train_df[['income', 'balance']]
y_train = train_df['default']
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

# Step (iii): Obtain predictions on the validation set
X_valid = valid_df[['income', 'balance']]
y_valid_pred_prob = logreg_model.predict_proba(X_valid)[:, 1]
y_valid_pred = np.where(y_valid_pred_prob > 0.5, 'Yes', 'No')  # Convert to string labels

# Step (iv): Compute the validation set error
validation_error = 1 - accuracy_score(valid_df['default'], y_valid_pred)

# Display the validation set error
print(f"Validation Set Error: {validation_error:.4f}")

(c)  Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [ ]:
# Number of repetitions
num_repeats = 3

for repeat in range(num_repeats):
    # Step (i): Split the dataset into training and validation sets
    train_df, valid_df = train_test_split(Default, test_size=0.2, random_state=16 * (repeat + 1))

    # Step (ii): Fit a logistic regression model using only the training observations
    X_train = train_df[['income', 'balance']]
    y_train = train_df['default']
    logreg_model = LogisticRegression()
    logreg_model.fit(X_train, y_train)

    # Step (iii): Obtain predictions on the validation set
    X_valid = valid_df[['income', 'balance']]
    y_valid_pred_prob = logreg_model.predict_proba(X_valid)[:, 1]
    y_valid_pred = np.where(y_valid_pred_prob > 0.5, 'Yes', 'No')  # Convert to string labels

    # Step (iv): Compute the validation set error
    validation_error = 1 - accuracy_score(valid_df['default'], y_valid_pred)

    # Display the validation set error for each iteration
    print(f"Validation Set Error (Repeat {repeat + 1}): {validation_error:.4f}")

(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [ ]:
# Number of repetitions
num_repeats = 3

for repeat in range(num_repeats):
    # Step (i): Split the dataset into training and validation sets
    train_df, valid_df = train_test_split(Default, test_size=0.2, random_state=16 * (repeat + 1))

    # Step (ii): Define a pipeline for preprocessing and logistic regression
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', ['income', 'balance']),
            ('cat', OneHotEncoder(), ['student'])
        ])

    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', LogisticRegression())])

    # Step (iii): Fit the logistic regression model using only the training observations
    X_train = train_df[['income', 'balance', 'student']]
    y_train = train_df['default']
    model.fit(X_train, y_train)

    # Step (iv): Obtain predictions on the validation set
    X_valid = valid_df[['income', 'balance', 'student']]
    y_valid_pred = model.predict(X_valid)

    # Step (v): Compute the validation set error
    validation_error = 1 - accuracy_score(valid_df['default'], y_valid_pred)

    # Display the validation set error for each iteration
    print(f"Validation Set Error (Repeat {repeat + 1}): {validation_error:.4f}")